In [1]:
import pandas as pd
import json

# Load the dataset
data = pd.read_csv('ev_car_problems.csv')

# Preprocess the dataset
data['Symptoms'] = data['Symptoms'].apply(json.loads)
data['Solutions'] = data['Solutions'].apply(json.loads)

print(data.head())


                Problem Name  \
0        Battery Degradation   
1  Charging Port Malfunction   
2          Motor Overheating   
3            Software Glitch   
4       Braking System Error   

                                            Symptoms  \
0  [Reduced battery capacity, Increased charging ...   
1  [Intermittent charging, Loose connection indic...   
2  [Increased temperature gauge reading, Coolant ...   
3  [Application crashes, Frozen screen, System er...   
4  [Spongy brake pedal, Brake warning light, Redu...   

                                           Solutions  
0  [Replace battery, Optimize charging habits, Ch...  
1  [Check port connections, Replace port if neces...  
2  [Check cooling system, Improve ventilation, Fl...  
3  [Update software, Reset system, Perform factor...  
4  [Check brake fluid, Inspect brake components, ...  


In [2]:
pip install google-cloud-dialogflow


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from google.cloud import dialogflow_v2 as dialogflow
import pandas as pd
import time
import json  # Importing json module

# Set own Google Application Credentials with Dialogflow  

# Load the dataset
data = pd.read_csv('ev_car_problems.csv')
data['Symptoms'] = data['Symptoms'].apply(json.loads)
data['Solutions'] = data['Solutions'].apply(json.loads)

# Function to create intents in Dialogflow
def create_intent(display_name, training_phrases_parts, message_texts):
    intents_client = dialogflow.IntentsClient()

    parent = f"projects/{project_id}/agent"

    training_phrases = []
    for training_phrase in training_phrases_parts:
        parts = []
        for part_text in training_phrase:
            part = dialogflow.Intent.TrainingPhrase.Part(text=part_text)
            parts.append(part)
        training_phrases.append(dialogflow.Intent.TrainingPhrase(parts=parts))

    text = dialogflow.Intent.Message.Text(text=message_texts)
    message = dialogflow.Intent.Message(text=text)

    intent = dialogflow.Intent(
        display_name=display_name,
        training_phrases=training_phrases,
        messages=[message],
    )

    response = intents_client.create_intent(parent=parent, intent=intent)

    print('Intent created: {}'.format(response))
delay_seconds = 1  # Delay in seconds

# Loop through the dataset and create intents
for index, row in data.iterrows():
    problem = row['Problem Name']
    symptoms = row['Symptoms']
    solutions = row['Solutions']
    create_intent(problem, symptoms, solutions)
    time.sleep(delay_seconds)

print("Intent creation process completed.")


Intent created: name: "projects/tata-429322/agent/intents/82bc0026-a674-48b8-a517-b60b57a00ebd"
display_name: "Battery Degradation"
priority: 500000
messages {
  text {
    text: "Replace battery"
    text: "Optimize charging habits"
    text: "Check for firmware updates"
  }
}

Intent created: name: "projects/tata-429322/agent/intents/eefb9d6c-fe85-463f-853f-bd97726de204"
display_name: "Charging Port Malfunction"
priority: 500000
messages {
  text {
    text: "Check port connections"
    text: "Replace port if necessary"
    text: "Clean port contacts"
  }
}

Intent created: name: "projects/tata-429322/agent/intents/8f071926-c11f-4ceb-9663-53e65493e9d4"
display_name: "Motor Overheating"
priority: 500000
messages {
  text {
    text: "Check cooling system"
    text: "Improve ventilation"
    text: "Flush cooling system"
  }
}

Intent created: name: "projects/tata-429322/agent/intents/7ba6ea55-106c-4d22-b861-bd0cea587daf"
display_name: "Software Glitch"
priority: 500000
messages {
  tex

In [8]:
import pandas as pd
import requests
from nltk.metrics.distance import edit_distance
# from nltk.corpus import wordnet

# Load the dataset
data = pd.read_csv('ev_car_problems.csv')

# # Function to get synonyms using WordNet
# def get_synonyms(word):
#     synonyms = set()
#     for synset in wordnet.synsets(word):
#         for lemma in synset.lemmas():
#             synonyms.add(lemma.name().replace('_', ' '))
#     return list(synonyms)

def get_related_terms(word):
    related_terms = set()
    url = f"http://api.conceptnet.io/c/en/{word}?offset=0&limit=1000"
    response = requests.get(url).json()

    for edge in response['edges']:
        related_terms.add(edge['end']['label'])

    return list(related_terms)

def find_closest_problem(symptoms_input):
    min_distance = float('inf')
    closest_problem = None

    for index, row in data.iterrows():
        symptoms = eval(row['Symptoms'])  # Evaluating JSON string to Python list
        for symptom in symptoms:
            distance = edit_distance(symptoms_input, symptom)
            if distance < min_distance:
                min_distance = distance
                closest_problem = row['Problem Name']

    return closest_problem

def get_solutions(problem_name):
    solutions = data[data['Problem Name'] == problem_name]['Solutions'].iloc[0]
    return eval(solutions)  # Evaluating JSON string to Python list

# Function to handle user input and interaction
def handle_user_input():
    attempts = 0
    
    while attempts < 3:
        user_symptoms = input("Please describe your symptoms: ").strip().lower()
        related_terms = get_related_terms(user_symptoms)
        closest_problem = find_closest_problem(user_symptoms)
        
        if closest_problem:
            # Present closest problem to user and ask for confirmation
            print(f"We identified the closest problem: {closest_problem}")
            user_response = input("Is this the problem you're experiencing? (yes/no): ").strip().lower()
            
            if user_response == 'yes':
                solutions = get_solutions(closest_problem)
                print(f"Solutions for {closest_problem}:")
                for solution in solutions:
                    print(f"- {solution}")
                break  # Exit loop on successful identification and confirmation
            else:
                attempts += 1
                print("Let's try to find another problem based on your input.")
        else:
            attempts += 1
            print("We couldn't match your symptoms to any specific problem. Please provide more details.")
    
    if attempts == 3:
        print("We couldn't identify the problem after three attempts. Please contact xyz@example.com for assistance.")

# Example usage
handle_user_input()


We identified the closest problem: Software Glitch
Let's try to find another problem based on your input.
We identified the closest problem: Software Glitch
Solutions for Software Glitch:
- Update software
- Reset system
- Perform factory reset
